In [62]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')

In [63]:
%%time

data = pd.read_csv('data/autok_final.csv.gz', compression = 'gzip')

Wall time: 3.48 s


### Correct data types

#### Clean column names

In [64]:
data.columns = [i.replace(':', '').strip() for i in data.columns]

#### Check missing values

In [65]:
NA_count = data.isnull().sum() / data.shape[0]
NA_count[NA_count > 1/3]

Extrákkal növelt ár       0.906094
Akciós ár                 0.912635
Akció feltételei          0.916953
Átvehető                  0.891906
Modellcsoport             0.728347
Fordulási kör átmérője    0.384454
Oktánszám                 0.522199
Garancia                  0.945147
Hasmagasság               0.816818
Első terepszög            0.900866
Hátsó terepszög           0.900775
Rámpaszög                 0.926797
Gázlómélység              0.958047
Alaptípus ára             0.935848
dtype: float64

Some of the price features will be used later, but the car features with most NaN can be dropped

In [66]:
drop_due_to_NA = ['Oktánszám', 'Fordulási kör átmérője', 'Hasmagasság', 'Gázlómélység', 'Garancia',
                  'Átvehető', 'Első terepszög', 'Hátsó terepszög', 'Rámpaszög', 'Akció feltételei']
data.drop(drop_due_to_NA, 1, inplace = True)

#### Check dtypes - clean columns of unnecessary strings (i.e: 50kW --> 50, 5 doors --> 5)

In [67]:
print(data.shape)
data = data[data['Vételár'] != 'Ár nélkül']
print(data.shape)

(76896, 42)
(76865, 42)


In [72]:
%%time

# has a comma between numbers, need to replace it with dot and make it a number
data['Gyorsulás 0-ról 100 km/h-ra'] = data['Gyorsulás 0-ról 100 km/h-ra'].str.replace(' s', '').str.strip().str.replace(',', '.').astype(float)

# others dont have commas
numeric_cols = ['Vételár', 'Vételár EUR', 'Extrákkal növelt ár', 'Akciós ár', 'Alaptípus ára',
                'Kilométeróra állása', 'Szállítható szem. száma', 'Ajtók száma', 'Saját tömeg', 
                'Teljes tömeg', 'Csomagtartó', 'Hengerűrtartalom', 'Szélesség', 'Magasság', 'Hosszúság', 'Tengelytáv', 
                'Üzemanyagtank', 'Hasznos teher', 'CO2-kibocsátás', 'Végsebesség']

# fogyasztas will need to be treated differently as there are two numbers there (5.6l/100km)

# for i in numeric_cols:
#     if i in data.columns:
#         if is_numeric_dtype(data[i]) == False:
#             print(data[i].str.replace(r'\d+', '').replace('', np.nan).value_counts(dropna = False))

# vetelar (HUF) has a couple of EUR values, treat them
data.loc[data['Vételár'].fillna('').str.contains('€'), 'Vételár'] = np.nan

for i in numeric_cols:
    if i in data.columns:
        if is_numeric_dtype(data[i]) == False:
            data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)

Wall time: 3.08 s


Remainind columns to be treated:
- Teljesítmény (55kW, 75LE --> split into two)
- Fogyasztások (városi, országúti, vegyes) (8,6 l/100km	--> 8.6)

In [73]:
data[['Teljesítmény kW', 'Teljesítmény LE']] = data['Teljesítmény'].str.split(',', expand = True)
data.drop('Teljesítmény', 1, inplace = True)

In [74]:
teljesitmeny = ['Teljesítmény kW', 'Teljesítmény LE']

for i in teljesitmeny:
    if i in data.columns:
        if is_numeric_dtype(data[i]) == False:
            data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)

In [75]:
fogyasztasok = data.filter(like = 'fogyasztás').columns

for i in fogyasztasok:
    if i in data.columns:
        data[i] = data[i].str.replace('l/100km', '').str.strip().str.replace(',', '.').astype(float)

#### Convert remaining string columns that should be numeric to floats
Évjárat:
- delete everything between paranthesis
- delete everything after first / sign
- convert string year to int year

Műszaki vizsga érvényes
- delete everything after first / sign
- convert string year to int year (Int64 due to NaNs)

In [76]:
data['Évjárat'] = data['Évjárat'].str.replace(r'\(.*\)', '').str.replace('(/).*', '').str.strip().astype(int)

In [77]:
data['Műszaki vizsga érvényes'] = data['Műszaki vizsga érvényes'].str.replace('(/).*', '').replace('', np.nan).str.strip().astype(float).astype('Int64')

### Check categorical features - normalize them, group them

In [78]:
data.select_dtypes('O').describe().T

,count,unique,top,freq
Állapot,76865,18,Normál,28003
Kivitel,76865,15,Ferdehátú,24108
Üzemanyag,76319,10,Benzin,40937
Hajtás,71319,5,Első kerék,51770
Sebességváltó fajtája,72332,49,Manuális (5 fokozatú),29050
Okmányok jellege,76854,7,Érvényes magyar okmányokkal,65511
URL,76865,76865,https://www.hasznaltauto.hu/szemelyauto/audi/a...,1
Hirdetés név,76865,56890,BMW 320d,76
Felszereltség,71933,65900,"{'Beltér': ['bőr belső', 'ülésmagasság állítás...",144
Kategória,68112,9,Alsó középkategória,19861


In [79]:
%%time

data.loc[data['Állapot'].str.lower().str.contains('sérült', na = False), 'Állapot'] = 'Sérült'
data.loc[data['Állapot'].str.lower().str.contains('hibás', na = False), 'Állapot'] = 'Sérült'
data.loc[data['Állapot'].str.lower().str.contains('hiány', na = False), 'Állapot'] = 'Sérült'

data = data[~data['Kivitel'].isin(['Buggy', 'Mopedautó', 'Egyéb', 'Sport'])]
data.loc[data['Kivitel'].str.lower().str.contains('terep', na = False), 'Kivitel'] = 'Terepjáró'
data.loc[data['Kivitel'].str.lower().str.contains('pickup', na = False), 'Kivitel'] = 'Terepjáró'
data.loc[data['Kivitel'].str.lower().str.contains('lépcsőshátú', na = False), 'Kivitel'] = 'Coupe'

data = data[~data['Üzemanyag'].isin(['Etanol', 'CNG', 'LPG'])]
data.loc[data['Üzemanyag'].str.lower().str.contains('hibrid', na = False), 'Üzemanyag'] = 'Hibrid'
data.loc[data['Üzemanyag'].str.lower().str.contains('benzin', na = False), 'Üzemanyag'] = 'Benzin'

data.loc[data['Hajtás'].str.lower().str.contains('össz', na = False), 'Hajtás'] = 'Összkerék'

data.loc[data['Okmányok jellege'].str.lower().str.contains('érvényes', na = False), 'Okmányok jellege'] = 'Érvényes'
data.loc[data['Okmányok jellege'].str.lower().str.contains('lejárt', na = False), 'Okmányok jellege'] = 'Lejárt'
data.loc[data['Okmányok jellege'].str.lower().str.contains('kivont', na = False), 'Okmányok jellege'] = 'Kivont'

data = data[data['Klíma fajtája'] != 'Hőszivattyús klíma']
data.loc[data['Klíma fajtája'].str.lower().str.contains('digitális', na = False), 'Klíma fajtája'] = 'Digitális'
data.loc[data['Klíma fajtája'].str.lower().str.contains('manuális', na = False), 'Klíma fajtája'] = 'Manuális'
data.loc[data['Klíma fajtája'].str.lower().str.contains('automata', na = False), 'Klíma fajtája'] = 'Automata'

data['Szín - metál'] = data['Szín'].apply(lambda x: 1 if 'metál' in str(x).lower() else 0)
data['Szín'] = data['Szín'].str.replace(r'\(.*\)', '').str.strip().replace('', np.nan)
data.loc[data['Szín'].str.lower().str.contains('szürke', na = False), 'Szín'] = 'Szürke'
data.loc[data['Szín'].str.lower().str.contains('kék', na = False), 'Szín'] = 'Kék'
data.loc[data['Szín'].str.lower().str.contains('narancs', na = False), 'Szín'] = 'Narancssárga'
data.loc[data['Szín'].str.lower().str.contains('sárga', na = False), 'Szín'] = 'Sárga'
data.loc[data['Szín'].str.lower().str.contains('okker', na = False), 'Szín'] = 'Sárga'
data.loc[data['Szín'].str.lower().str.contains('fehér', na = False), 'Szín'] = 'Fehér'
data.loc[data['Szín'].str.lower().str.contains('barna', na = False), 'Szín'] = 'Barna'
data.loc[data['Szín'].str.lower().str.contains('zöld', na = False), 'Szín'] = 'Zöld'
data.loc[data['Szín'].str.lower().str.contains('türkiz', na = False), 'Szín'] = 'Kék'
top_szinek = ['Fekete', 'Szürke', 'Fehér', 'Ezüst', 'Kék', 'Piros', 'Barna', 'Zöld', 'Bordó', 'Sárga']
data.loc[~data['Szín'].isin(top_szinek) & data['Szín'].notnull(), 'Szín'] = 'Egyéb'

data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('automata', na = False), 'Sebességváltó fajtája'] = 'Automata'
data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('szekvenciális', na = False), 'Sebességváltó fajtája'] = 'Automata'
data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('manuális', na = False), 'Sebességváltó fajtája'] = 'Manuális'
data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('tiptronic', na = False), 'Sebességváltó fajtája'] = 'Automata'

Wall time: 3.15 s


In [80]:
marka_count = data['Márka'].value_counts()
drop_brands = marka_count[marka_count < 300].index.tolist()

drop_brands = ['SMART', 'JAGUAR', 'LADA', 'CHRYSLER', 'MERCEDES-AMG', 'SSANGYONG', 'DAEWOO', 'SUBARU', 'TESLA', 'LANCIA', 'DODGE', 'INFINITI', 'SAAB', 'DAIHATSU', 'TRABANT', 'MASERATI', 'ROVER', 'CADILLAC', 'ABARTH', 'WARTBURG', 'CUPRA', 'LINCOLN', 'FERRARI', 'POLSKI FIAT', 'BENTLEY', 'VERSENYAUTÓ', 'ISUZU', 'MG', 'PONTIAC', 'HUMMER', 'ASTON MARTIN', 'GAZ', 'UAZ', 'ROLLS-ROYCE', 'MARUTI', 'DS', 'BUICK', 'TATA', 'BLUECAR', 'GMC', 'ZASTAVA', 'VOLGA', 'MOSZKVICS', 'IVECO', 'ACURA', 'OLDSMOBILE', 'ALPINA', 'MAHINDRA', 'LAMBORGHINI', 'FORD (USA)', 'NSU', 'MATRA', 'DATSUN', 'FISKER', 'YUGO', 'LOTUS', 'BARKAS', 'POLESTAR', 'AIXAM', 'GAC GONOW', 'GREAT WALL', 'DAIMLER', 'INNOCENTI', 'PROTON', 'DKW', 'PLYMOUTH', 'LUAZ', 'AUTOBIANCHI', 'TAZZARI', 'DUTTON', 'ARO', 'BERTONE', 'MERCURY', 'WARSZAWA', 'BRILLIANCE (BMW)', 'ADLER', 'WOLSELEY', 'ASIA', 'REPLIKA', 'POLARIS', 'TRIUMPH', 'HUDSON', 'FORD (AUSTRALIA)', 'PACKARD', 'GEO', 'NASH', 'MORGAN', 'SVW (VOLKSWAGEN)', 'ZAPOROZSEC', 'LIGIER', 'EXCALIBUR', 'EGYEDI', 'MERCEDES-MAYBACH', 'GOGGOMOBIL', 'BORGWARD', 'TALBOT', 'BRILLIANCE (ZHONGHUA)', 'AUSTIN']
print('Dropping the following brands due to low count:')
print(drop_brands)

data = data[~data['Márka'].isin(drop_brands)]

Dropping the following brands due to low count:
['SMART', 'JAGUAR', 'LADA', 'CHRYSLER', 'MERCEDES-AMG', 'SSANGYONG', 'DAEWOO', 'SUBARU', 'TESLA', 'LANCIA', 'DODGE', 'INFINITI', 'SAAB', 'DAIHATSU', 'TRABANT', 'MASERATI', 'ROVER', 'CADILLAC', 'ABARTH', 'WARTBURG', 'CUPRA', 'LINCOLN', 'FERRARI', 'POLSKI FIAT', 'BENTLEY', 'VERSENYAUTÓ', 'ISUZU', 'MG', 'PONTIAC', 'HUMMER', 'ASTON MARTIN', 'GAZ', 'UAZ', 'ROLLS-ROYCE', 'MARUTI', 'DS', 'BUICK', 'TATA', 'BLUECAR', 'GMC', 'ZASTAVA', 'VOLGA', 'MOSZKVICS', 'IVECO', 'ACURA', 'OLDSMOBILE', 'ALPINA', 'MAHINDRA', 'LAMBORGHINI', 'FORD (USA)', 'NSU', 'MATRA', 'DATSUN', 'FISKER', 'YUGO', 'LOTUS', 'BARKAS', 'POLESTAR', 'AIXAM', 'GAC GONOW', 'GREAT WALL', 'DAIMLER', 'INNOCENTI', 'PROTON', 'DKW', 'PLYMOUTH', 'LUAZ', 'AUTOBIANCHI', 'TAZZARI', 'DUTTON', 'ARO', 'BERTONE', 'MERCURY', 'WARSZAWA', 'BRILLIANCE (BMW)', 'ADLER', 'WOLSELEY', 'ASIA', 'REPLIKA', 'POLARIS', 'TRIUMPH', 'HUDSON', 'FORD (AUSTRALIA)', 'PACKARD', 'GEO', 'NASH', 'MORGAN', 'SVW (VOLKSWAGEN)', 

Márka is too broad, modell is too low-level. Using Modellcsoport

In [81]:
%%time

data['Modellcsoport modellezéshez'] = data['Modellcsoport'].copy()

modell_count = data['Modell'].value_counts()
modell_count = modell_count[modell_count >= 150].index.tolist()
modell_count = ['FOCUS', 'OCTAVIA', 'A4', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', 'ASTRA H', 'FIESTA', '320', "CEE'D", 'YARIS', 'A3', 'CLIO', 'FABIA', 'I30', 'CIVIC', '6', 'ASTRA J', 'QASHQAI', '500', 'COROLLA', 'PASSAT', 'CORSA D', 'ASTRA G', '3', 'INSIGNIA', 'X5', '530', 'SUPERB', 'KUGA', 'JAZZ', 'C-MAX', 'POLO', 'ASTRA K', 'AVENSIS', '206', '318', 'GOLF VII', 'VITARA', 'PASSAT VIII', 'PUNTO', '520', 'AURIS', 'GOLF V', 'LEON', 'PUMA', 'SCENIC', 'X3', 'TRANSIT', 'IBIZA', 'E 220', 'MICRA', 'IGNIS', 'GOLF VI', 'SPORTAGE', '308', 'TOURAN', 'SX4', 'TIPO', 'C3', 'CORSA F', 'X-TRAIL', 'A5', 'C 220', 'Q7', 'JUKE', 'RAV 4', 'S-MAX', 'ZAFIRA B', 'LAGUNA', 'X6', '307', 'GOLF IV', 'TIGUAN', '207', 'I3', '330', '2', 'MERIVA A', 'DUSTER', 'XC60', 'GALAXY', 'I20', 'Q5', 'ACCORD', '525', 'GOLF', 'LEAF', 'XSARA PICASSO', 'TUCSON', 'C4', 'SANDERO', 'MERIVA', '116', 'C 200', 'CORSA C', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'TOUAREG', 'C5', 'OUTLANDER', 'GRANDE PUNTO', 'ALTEA', 'PASSAT VI', 'A8', '316', 'COLT', '5008', '3008', 'E 200', 'CAYENNE', '5', 'XC90', 'SORENTO', 'ASTRA', 'GOLF PLUS', 'ASTRA F', 'PANDA', 'TWINGO', 'RIO', '3-AS SOROZAT', '159', 'VECTRA C', 'ECOSPORT', '208', 'AYGO', 'ESPACE', 'MOKKA', 'PASSAT V', 'X1', 'S60', 'CORSA', 'C 180', 'POLO IV']
data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modellcsoport modellezéshez'] = data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modell']

modellcsoport_count = data['Modellcsoport modellezéshez'].value_counts()
modellcsoport_count = modellcsoport_count[modellcsoport_count >= 150].index.tolist()
modellcsoport_count = ['ASTRA', '3-AS SOROZAT', 'FOCUS', 'GOLF', 'PASSAT', 'X SOROZAT', '5-ÖS SOROZAT', 'OCTAVIA', 'CORSA', 'E-OSZTÁLY', 'A4', 'C-OSZTÁLY', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', "CEE'D", 'FIESTA', 'POLO', 'YARIS', 'A3', 'COROLLA', 'CLIO', 'FABIA', 'ZAFIRA', 'I30', 'MERIVA', 'A-OSZTÁLY', 'CIVIC', '6', 'QASHQAI', 'PUNTO', '1-ES SOROZAT', '500', '3', 'SCENIC', 'INSIGNIA', 'SUPERB', 'KUGA', 'JAZZ', 'AVENSIS', 'VITARA', 'S-OSZTÁLY', 'C-MAX', '206', '7-ES SOROZAT', 'AURIS', 'LEON', 'PUMA', 'B-OSZTÁLY', 'MICRA', 'TRANSIT', 'IBIZA', 'A5', 'C4 PICASSO', 'IGNIS', 'SPORTAGE', 'XSARA', '308', 'TOURAN', 'SX4', 'TIPO', 'VECTRA', 'C3', 'X-TRAIL', 'I SOROZAT', 'Q7', 'JUKE', 'TIGUAN', 'RAV 4', 'S-MAX', 'LAGUNA', 'MINI', 'JETTA', '307', '207', '2', 'DUSTER', 'XC60', 'I20', 'GALAXY', 'ACCORD', 'Q5', 'ML-OSZTÁLY', 'LEAF', 'GLE-OSZTÁLY', 'TUCSON', 'SANDERO', 'C4', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'ESPACE', 'TOUAREG', 'C5', 'OUTLANDER', 'ALTEA', 'A8', 'GLC-OSZTÁLY', 'M SOROZAT', 'COLT', '5008', 'CAYENNE', '3008', 'CLA-OSZTÁLY', '5', 'XC90', 'SORENTO', '4-ES SOROZAT', 'PRIUS', 'TRANSPORTER', 'PANDA', 'TWINGO', 'RIO', '159', 'ECOSPORT', 'AYGO', '208', 'MOKKA', 'S60']
data['Modellcsoport modellezéshez'] = data['Modellcsoport modellezéshez'].apply(lambda x: 'EGYÉB' if x not in modellcsoport_count else x)

data['Modellcsoport modellezéshez'] = data['Márka'] + ' ' + data['Modellcsoport modellezéshez']
data.loc[data['Modellcsoport modellezéshez'] == 'MERCEDES-BENZ 500', 'Modellcsoport modellezéshez'] = 'MERCEDES-BENZ EGYÉB'

Wall time: 254 ms


#### Expand felszereltség dictionaty into 4 columns

In [82]:
%%time

data['Felszereltség'] = data['Felszereltség'].apply(lambda x: literal_eval(x) if isinstance(x, float) == False else {})
felszereltseg_expanded = data['Felszereltség'].apply(pd.Series)
data = pd.concat([data, felszereltseg_expanded], 1)
data.drop('Felszereltség', 1, inplace = True)

Wall time: 37.4 s


After cleaning and normalizing categories

In [21]:
%%time

data.select_dtypes('O').describe().T

Wall time: 5min 6s


,count,unique,top,freq
Állapot,72551,6,Normál,26302
Kivitel,72551,8,Ferdehátú,23184
Üzemanyag,72188,4,Benzin,38703
Hajtás,67770,3,Első kerék,50107
Sebességváltó fajtája,68513,2,Manuális,48446
Okmányok jellege,72543,4,Érvényes,63525
URL,72551,72551,https://www.hasznaltauto.hu/szemelyauto/audi/a...,1
Hirdetés név,72551,53627,BMW 320d,76
Kategória,65300,9,Alsó középkategória,19222
Márka,72551,29,OPEL,6758


### Check numeric features - distributions, correlations

In [83]:
data.select_dtypes(exclude = 'O').describe(percentiles = [.01, .1, .25, .5, .75, .9, .99]).T.astype(int)

,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
Extrákkal növelt ár,6957,11607687,7090021,3799000,4460000,5890000,7286000,9174800,13359300,20585172,38401666,78905000
Akciós ár,6489,9642420,5952661,220000,1799000,5050000,6140000,7860000,10990000,17000000,33106199,66090000
Vételár EUR,72541,14137,19852,55,491,1698,3329,6934,16615,33466,101492,193304
Évjárat,72551,2010,7,1918,1989,2002,2006,2011,2017,2021,2021,2021
Kilométeróra állása,70498,156084,141537,1,1,6000,76000,155896,221137,289718,427218,16777215
Szállítható szem. száma,71321,5,2,1,4,5,5,5,5,5,8,255
Ajtók száma,70816,4,0,2,2,3,4,5,5,5,5,5
Saját tömeg,66236,1409,332,95,810,1030,1183,1370,1585,1845,2373,12340
Teljes tömeg,66018,1952,430,10,1250,1495,1690,1897,2150,2500,3160,31000
Csomagtartó,62384,444,352,1,139,250,327,435,514,586,1030,15500


In [90]:
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény LE'] = 80
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény kW'] = 59
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény LE'] = 84
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény kW'] = 62
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény LE'] = 306
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény kW'] = 225
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény LE'] = 70
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény kW'] = 52

data['Saját tömeg'] = data['Saját tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)
data['Teljes tömeg'] = data['Teljes tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)

data = data[data['Üzemanyagtank'] <= 120]
data = data[data['Hengerűrtartalom'] < 10000]
data = data[data['Szállítható szem. száma'] < 20]
data = data[data['Kilométeróra állása'] < 1000000]

After cleaning some extremes or mistakes

In [104]:
data.select_dtypes(exclude = 'O').describe(percentiles = [.01, .1, .25, .5, .75, .9, .99]).T.astype(int)

,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
Extrákkal növelt ár,3579,11170813,6730640,3865000,4410000,5828000,7153460,8710000,12915280,19700000,36958512,78905000
Akciós ár,3412,9217937,5828352,220000,1255390,4985400,5978200,7390000,10450000,16190000,32787899,57990000
Vételár EUR,59537,10829,14508,55,485,1659,3051,5941,12759,24021,74614,191390
Évjárat,59544,2010,6,1975,1996,2002,2006,2010,2015,2019,2021,2021
Kilométeróra állása,59544,165826,100063,1,1,18827,99500,165706,227000,292000,424000,990000
Szállítható szem. száma,59544,5,0,2,4,5,5,5,5,5,8,12
Ajtók száma,59406,4,0,2,2,3,4,5,5,5,5,5
Saját tömeg,58763,1402,324,12,810,1029,1180,1360,1579,1840,2348,3500
Teljes tömeg,58654,1945,391,10,1252,1495,1685,1890,2140,2495,3140,4430
Csomagtartó,56747,444,355,1,139,250,325,435,513,585,1010,15500


### All cleaning in 1 function

In [108]:
def cleaner(data):
    
    data.columns = [i.replace(':', '').strip() for i in data.columns]
    
    drop_due_to_NA = ['Oktánszám', 'Fordulási kör átmérője', 'Hasmagasság', 'Gázlómélység', 'Garancia',
                  'Átvehető', 'Első terepszög', 'Hátsó terepszög', 'Rámpaszög', 'Akció feltételei']
    for i in drop_due_to_NA:
        if i in data.columns:
            data.drop(i, 1, inplace = True)

    data = data[data['Vételár'] != 'Ár nélkül']
    data.loc[data['Vételár'].fillna('').str.contains('€'), 'Vételár'] = np.nan
    
    numeric_cols = ['Vételár', 'Vételár EUR', 'Extrákkal növelt ár', 'Akciós ár', 'Alaptípus ára',
                    'Kilométeróra állása', 'Szállítható szem. száma', 'Ajtók száma', 'Saját tömeg', 
                    'Teljes tömeg', 'Csomagtartó', 'Hengerűrtartalom', 'Szélesség', 'Magasság', 'Hosszúság', 'Tengelytáv', 
                    'Üzemanyagtank', 'Hasznos teher', 'CO2-kibocsátás', 'Végsebesség', 'Gyorsulás 0-ról 100 km/h-ra']
    for i in numeric_cols:
        if i in data.columns:
            if is_numeric_dtype(data[i]) == False:
                data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)
                
    data[['Teljesítmény kW', 'Teljesítmény LE']] = data['Teljesítmény'].str.split(',', expand = True)
    data.drop('Teljesítmény', 1, inplace = True)
    teljesitmeny = ['Teljesítmény kW', 'Teljesítmény LE']
    for i in teljesitmeny:
        if i in data.columns:
            if is_numeric_dtype(data[i]) == False:
                data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)
    
    fogyasztasok = data.filter(like = 'fogyasztás').columns

    for i in fogyasztasok:
        if i in data.columns:
            data[i] = data[i].str.replace('l/100km', '').str.strip().str.replace(',', '.').astype(float)
            
    data['Évjárat'] = data['Évjárat'].str.replace(r'\(.*\)', '').str.replace('(/).*', '').str.strip().astype(int)
    data['Műszaki vizsga érvényes'] = data['Műszaki vizsga érvényes'].str.replace('(/).*', '').replace('', np.nan).str.strip().astype(float).astype('Int64')
    
    data.loc[data['Állapot'].str.lower().str.contains('sérült', na = False), 'Állapot'] = 'Sérült'
    data.loc[data['Állapot'].str.lower().str.contains('hibás', na = False), 'Állapot'] = 'Sérült'
    data.loc[data['Állapot'].str.lower().str.contains('hiány', na = False), 'Állapot'] = 'Sérült'

    data = data[~data['Kivitel'].isin(['Buggy', 'Mopedautó', 'Egyéb', 'Sport'])]
    data.loc[data['Kivitel'].str.lower().str.contains('terep', na = False), 'Kivitel'] = 'Terepjáró'
    data.loc[data['Kivitel'].str.lower().str.contains('pickup', na = False), 'Kivitel'] = 'Terepjáró'
    data.loc[data['Kivitel'].str.lower().str.contains('lépcsőshátú', na = False), 'Kivitel'] = 'Coupe'

    data = data[~data['Üzemanyag'].isin(['Etanol', 'CNG', 'LPG'])]
    data.loc[data['Üzemanyag'].str.lower().str.contains('hibrid', na = False), 'Üzemanyag'] = 'Hibrid'
    data.loc[data['Üzemanyag'].str.lower().str.contains('benzin', na = False), 'Üzemanyag'] = 'Benzin'

    data.loc[data['Hajtás'].str.lower().str.contains('össz', na = False), 'Hajtás'] = 'Összkerék'

    data.loc[data['Okmányok jellege'].str.lower().str.contains('érvényes', na = False), 'Okmányok jellege'] = 'Érvényes'
    data.loc[data['Okmányok jellege'].str.lower().str.contains('lejárt', na = False), 'Okmányok jellege'] = 'Lejárt'
    data.loc[data['Okmányok jellege'].str.lower().str.contains('kivont', na = False), 'Okmányok jellege'] = 'Kivont'

    data = data[data['Klíma fajtája'] != 'Hőszivattyús klíma']
    data.loc[data['Klíma fajtája'].str.lower().str.contains('digitális', na = False), 'Klíma fajtája'] = 'Digitális'
    data.loc[data['Klíma fajtája'].str.lower().str.contains('manuális', na = False), 'Klíma fajtája'] = 'Manuális'
    data.loc[data['Klíma fajtája'].str.lower().str.contains('automata', na = False), 'Klíma fajtája'] = 'Automata'

    data['Szín - metál'] = data['Szín'].apply(lambda x: 1 if 'metál' in str(x).lower() else 0)
    data['Szín'] = data['Szín'].str.replace(r'\(.*\)', '').str.strip().replace('', np.nan)
    data.loc[data['Szín'].str.lower().str.contains('szürke', na = False), 'Szín'] = 'Szürke'
    data.loc[data['Szín'].str.lower().str.contains('kék', na = False), 'Szín'] = 'Kék'
    data.loc[data['Szín'].str.lower().str.contains('narancs', na = False), 'Szín'] = 'Narancssárga'
    data.loc[data['Szín'].str.lower().str.contains('sárga', na = False), 'Szín'] = 'Sárga'
    data.loc[data['Szín'].str.lower().str.contains('okker', na = False), 'Szín'] = 'Sárga'
    data.loc[data['Szín'].str.lower().str.contains('fehér', na = False), 'Szín'] = 'Fehér'
    data.loc[data['Szín'].str.lower().str.contains('barna', na = False), 'Szín'] = 'Barna'
    data.loc[data['Szín'].str.lower().str.contains('zöld', na = False), 'Szín'] = 'Zöld'
    data.loc[data['Szín'].str.lower().str.contains('türkiz', na = False), 'Szín'] = 'Kék'
    top_szinek = ['Fekete', 'Szürke', 'Fehér', 'Ezüst', 'Kék', 'Piros', 'Barna', 'Zöld', 'Bordó', 'Sárga']
    data.loc[~data['Szín'].isin(top_szinek) & data['Szín'].notnull(), 'Szín'] = 'Egyéb'

    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('automata', na = False), 'Sebességváltó fajtája'] = 'Automata'
    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('szekvenciális', na = False), 'Sebességváltó fajtája'] = 'Automata'
    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('manuális', na = False), 'Sebességváltó fajtája'] = 'Manuális'
    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('tiptronic', na = False), 'Sebességváltó fajtája'] = 'Automata'

    drop_brands = ['SMART', 'JAGUAR', 'LADA', 'CHRYSLER', 'MERCEDES-AMG', 'SSANGYONG', 'DAEWOO', 'SUBARU', 'TESLA', 'LANCIA', 'DODGE', 'INFINITI', 'SAAB', 'DAIHATSU', 'TRABANT', 'MASERATI', 'ROVER', 'CADILLAC', 'ABARTH', 'WARTBURG', 'CUPRA', 'LINCOLN', 'FERRARI', 'POLSKI FIAT', 'BENTLEY', 'VERSENYAUTÓ', 'ISUZU', 'MG', 'PONTIAC', 'HUMMER', 'ASTON MARTIN', 'GAZ', 'UAZ', 'ROLLS-ROYCE', 'MARUTI', 'DS', 'BUICK', 'TATA', 'BLUECAR', 'GMC', 'ZASTAVA', 'VOLGA', 'MOSZKVICS', 'IVECO', 'ACURA', 'OLDSMOBILE', 'ALPINA', 'MAHINDRA', 'LAMBORGHINI', 'FORD (USA)', 'NSU', 'MATRA', 'DATSUN', 'FISKER', 'YUGO', 'LOTUS', 'BARKAS', 'POLESTAR', 'AIXAM', 'GAC GONOW', 'GREAT WALL', 'DAIMLER', 'INNOCENTI', 'PROTON', 'DKW', 'PLYMOUTH', 'LUAZ', 'AUTOBIANCHI', 'TAZZARI', 'DUTTON', 'ARO', 'BERTONE', 'MERCURY', 'WARSZAWA', 'BRILLIANCE (BMW)', 'ADLER', 'WOLSELEY', 'ASIA', 'REPLIKA', 'POLARIS', 'TRIUMPH', 'HUDSON', 'FORD (AUSTRALIA)', 'PACKARD', 'GEO', 'NASH', 'MORGAN', 'SVW (VOLKSWAGEN)', 'ZAPOROZSEC', 'LIGIER', 'EXCALIBUR', 'EGYEDI', 'MERCEDES-MAYBACH', 'GOGGOMOBIL', 'BORGWARD', 'TALBOT', 'BRILLIANCE (ZHONGHUA)', 'AUSTIN']
    print('Dropping the following brands due to low count:')
    print(drop_brands)
    data = data[~data['Márka'].isin(drop_brands)]
    
    modellcsoport_used = ['AUDI', 'BMW', 'CITROEN', 'FIAT', 'FORD', 'HYUNDAI', 'JEEP', 'KIA', 'MAZDA', 'MERCEDES-BENZ', 'MINI', 'MITSUBISHI', 'NISSAN', 'OPEL', 'RENAULT', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO']
    data.loc[(data['Márka'].isin(modellcsoport_used)) & (data['Modellcsoport'].isnull()), 'Modellcsoport'] = data.loc[(data['Márka'].isin(modellcsoport_used)) & (data['Modellcsoport'].isnull()), 'Modell']

    data['Modellcsoport modellezéshez'] = data['Modellcsoport'].copy()

    modell_count = ['FOCUS', 'OCTAVIA', 'A4', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', 'ASTRA H', 'FIESTA', '320', "CEE'D", 'YARIS', 'A3', 'CLIO', 'FABIA', 'I30', 'CIVIC', '6', 'ASTRA J', 'QASHQAI', '500', 'COROLLA', 'PASSAT', 'CORSA D', 'ASTRA G', '3', 'INSIGNIA', 'X5', '530', 'SUPERB', 'KUGA', 'JAZZ', 'C-MAX', 'POLO', 'ASTRA K', 'AVENSIS', '206', '318', 'GOLF VII', 'VITARA', 'PASSAT VIII', 'PUNTO', '520', 'AURIS', 'GOLF V', 'LEON', 'PUMA', 'SCENIC', 'X3', 'TRANSIT', 'IBIZA', 'E 220', 'MICRA', 'IGNIS', 'GOLF VI', 'SPORTAGE', '308', 'TOURAN', 'SX4', 'TIPO', 'C3', 'CORSA F', 'X-TRAIL', 'A5', 'C 220', 'Q7', 'JUKE', 'RAV 4', 'S-MAX', 'ZAFIRA B', 'LAGUNA', 'X6', '307', 'GOLF IV', 'TIGUAN', '207', 'I3', '330', '2', 'MERIVA A', 'DUSTER', 'XC60', 'GALAXY', 'I20', 'Q5', 'ACCORD', '525', 'GOLF', 'LEAF', 'XSARA PICASSO', 'TUCSON', 'C4', 'SANDERO', 'MERIVA', '116', 'C 200', 'CORSA C', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'TOUAREG', 'C5', 'OUTLANDER', 'GRANDE PUNTO', 'ALTEA', 'PASSAT VI', 'A8', '316', 'COLT', '5008', '3008', 'E 200', 'CAYENNE', '5', 'XC90', 'SORENTO', 'ASTRA', 'GOLF PLUS', 'ASTRA F', 'PANDA', 'TWINGO', 'RIO', '3-AS SOROZAT', '159', 'VECTRA C', 'ECOSPORT', '208', 'AYGO', 'ESPACE', 'MOKKA', 'PASSAT V', 'X1', 'S60', 'CORSA', 'C 180', 'POLO IV']
    data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modellcsoport modellezéshez'] = data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modell']

    modellcsoport_count = ['ASTRA', '3-AS SOROZAT', 'FOCUS', 'GOLF', 'PASSAT', 'X SOROZAT', '5-ÖS SOROZAT', 'OCTAVIA', 'CORSA', 'E-OSZTÁLY', 'A4', 'C-OSZTÁLY', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', "CEE'D", 'FIESTA', 'POLO', 'YARIS', 'A3', 'COROLLA', 'CLIO', 'FABIA', 'ZAFIRA', 'I30', 'MERIVA', 'A-OSZTÁLY', 'CIVIC', '6', 'QASHQAI', 'PUNTO', '1-ES SOROZAT', '500', '3', 'SCENIC', 'INSIGNIA', 'SUPERB', 'KUGA', 'JAZZ', 'AVENSIS', 'VITARA', 'S-OSZTÁLY', 'C-MAX', '206', '7-ES SOROZAT', 'AURIS', 'LEON', 'PUMA', 'B-OSZTÁLY', 'MICRA', 'TRANSIT', 'IBIZA', 'A5', 'C4 PICASSO', 'IGNIS', 'SPORTAGE', 'XSARA', '308', 'TOURAN', 'SX4', 'TIPO', 'VECTRA', 'C3', 'X-TRAIL', 'I SOROZAT', 'Q7', 'JUKE', 'TIGUAN', 'RAV 4', 'S-MAX', 'LAGUNA', 'MINI', 'JETTA', '307', '207', '2', 'DUSTER', 'XC60', 'I20', 'GALAXY', 'ACCORD', 'Q5', 'ML-OSZTÁLY', 'LEAF', 'GLE-OSZTÁLY', 'TUCSON', 'SANDERO', 'C4', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'ESPACE', 'TOUAREG', 'C5', 'OUTLANDER', 'ALTEA', 'A8', 'GLC-OSZTÁLY', 'M SOROZAT', 'COLT', '5008', 'CAYENNE', '3008', 'CLA-OSZTÁLY', '5', 'XC90', 'SORENTO', '4-ES SOROZAT', 'PRIUS', 'TRANSPORTER', 'PANDA', 'TWINGO', 'RIO', '159', 'ECOSPORT', 'AYGO', '208', 'MOKKA', 'S60']
    data['Modellcsoport modellezéshez'] = data['Modellcsoport modellezéshez'].apply(lambda x: 'EGYÉB' if x not in modellcsoport_count else x)

    data['Modellcsoport modellezéshez'] = data['Márka'] + ' ' + data['Modellcsoport modellezéshez']
    data.loc[data['Modellcsoport modellezéshez'] == 'MERCEDES-BENZ 500', 'Modellcsoport modellezéshez'] = 'MERCEDES-BENZ EGYÉB'
    
    data['Felszereltség'] = data['Felszereltség'].apply(lambda x: literal_eval(x) if isinstance(x, float) == False else {})
    felszereltseg_expanded = data['Felszereltség'].apply(pd.Series)
    data = pd.concat([data, felszereltseg_expanded], 1)
    data.drop('Felszereltség', 1, inplace = True)
    
    # numerical corrections
    
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény LE'] = 80
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény kW'] = 59
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény LE'] = 84
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény kW'] = 62
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény LE'] = 306
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény kW'] = 225
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény LE'] = 70
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény kW'] = 52

    data['Saját tömeg'] = data['Saját tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)
    data['Teljes tömeg'] = data['Teljes tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)

    data = data[data['Üzemanyagtank'] <= 120]
    data = data[data['Hengerűrtartalom'] < 10000]
    data = data[data['Szállítható szem. száma'] < 20]
    data = data[data['Kilométeróra állása'] < 1000000]
    
    return data

In [109]:
%%time

data = pd.read_csv('data/autok_final.csv.gz', compression = 'gzip')

Wall time: 3.38 s


In [110]:
%%time

print(data.shape)
data = cleaner(data)
print(data.shape)

(76896, 52)
Dropping the following brands due to low count:
['SMART', 'JAGUAR', 'LADA', 'CHRYSLER', 'MERCEDES-AMG', 'SSANGYONG', 'DAEWOO', 'SUBARU', 'TESLA', 'LANCIA', 'DODGE', 'INFINITI', 'SAAB', 'DAIHATSU', 'TRABANT', 'MASERATI', 'ROVER', 'CADILLAC', 'ABARTH', 'WARTBURG', 'CUPRA', 'LINCOLN', 'FERRARI', 'POLSKI FIAT', 'BENTLEY', 'VERSENYAUTÓ', 'ISUZU', 'MG', 'PONTIAC', 'HUMMER', 'ASTON MARTIN', 'GAZ', 'UAZ', 'ROLLS-ROYCE', 'MARUTI', 'DS', 'BUICK', 'TATA', 'BLUECAR', 'GMC', 'ZASTAVA', 'VOLGA', 'MOSZKVICS', 'IVECO', 'ACURA', 'OLDSMOBILE', 'ALPINA', 'MAHINDRA', 'LAMBORGHINI', 'FORD (USA)', 'NSU', 'MATRA', 'DATSUN', 'FISKER', 'YUGO', 'LOTUS', 'BARKAS', 'POLESTAR', 'AIXAM', 'GAC GONOW', 'GREAT WALL', 'DAIMLER', 'INNOCENTI', 'PROTON', 'DKW', 'PLYMOUTH', 'LUAZ', 'AUTOBIANCHI', 'TAZZARI', 'DUTTON', 'ARO', 'BERTONE', 'MERCURY', 'WARSZAWA', 'BRILLIANCE (BMW)', 'ADLER', 'WOLSELEY', 'ASIA', 'REPLIKA', 'POLARIS', 'TRIUMPH', 'HUDSON', 'FORD (AUSTRALIA)', 'PACKARD', 'GEO', 'NASH', 'MORGAN', 'SVW (VO

In [113]:
data.to_csv('data/autok_final_cleaned.csv.gz', compression = 'gzip', index = False)